In [ ]:
# ENVIRONMENT

from aisurveywriter.core.llm_handler import LLMHandler
import aisurveywriter.core.file_handler as fh
from aisurveywriter.utils import get_all_files_from_paths
from aisurveywriter.core.pipeline import PaperPipeline
from aisurveywriter.core.paper import PaperData
import aisurveywriter.tasks as tks

import os
os.environ["GOOGLE_API_KEY"]=fh.read_credentials("../credentials.yaml")["google_key"]

# llm = LLMHandler(model="qwen2.5:14b", model_type="ollama", temperature=0.5)
prompts = fh.read_yaml("../templates/prompt_config.yaml")
review = fh.read_yaml("../templates/review_config.yaml")

In [ ]:
from aisurveywriter.core.agent_rags import ImageData
from pydantic.json import pydantic_encoder
import json
import os

with open("../bib/figures_info.json", "r", encoding="utf-8") as f:
    objs = json.load(f)
figures = [ImageData(**obj) for obj in objs]

to_delete = []
for i, fig in enumerate(figures):
    if not os.path.isfile("../bib/filteredimgs/" + fig.basename):
        to_delete.append(i)
        print("Deleting", fig.basename)

for i in to_delete[::-1]:
    figures.pop(i)

for i in range(len(figures)):
    figures[i].id = i

with open("fitlered.json", "w", encoding="utf-8")  as f:
    json.dump(figures, f, default=pydantic_encoder)

In [ ]:
print(len(objs)-len(to_delete))

In [ ]:
from aisurveywriter.core.agent_rags import AgentRAG
from aisurveywriter.core.text_embedding import EmbeddingsHandler
import json

with open("../bib/figures_filtered.json", "r", encoding="utf-8") as f:
    objs = json.load(f)
figures = [ImageData(**obj) for obj in objs]

rag = AgentRAG(EmbeddingsHandler("Snowflake/snowflake-arctic-embed-l-v2.0", "huggingface"))
rag.create_faiss(rag._embed, figures, save_path="../bib/snowflake-figures-rag.faiss")